In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import glob
import os
import metpy
import numpy as np
from datetime import datetime
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
from metpy.calc import specific_humidity_from_dewpoint
from pint import UnitRegistry
ureg = UnitRegistry()
import seaborn as sns

In [29]:
# Function to extract epoch from the first few rows of the CSV file
def extract_epoch_from_header(file):
    # Read the first few lines to find the epoch (assuming it's in a comment or a header)
    with open(file, 'r') as f:
        skiprow = f.readline()
        skiprow = skiprow.strip().split()[0]
        for line in f:
            if "startdate:" in line.lower():  # Assuming the word 'epoch' is present in the line
                # Extract the epoch date (this assumes the date is the second item in the line)
                epoch = line.strip().split()[1]
                return int(skiprow)-1, epoch[0:4]+"-"+epoch[4:6]+"-"+epoch[6:8]+" "+epoch[8:10]+":"+epoch[10:12]+":"+epoch[12:14]
    return None  # Return None if no epoch is found

# Function to load and adjust time for each CSV
def load_and_adjust_time(file):
    # Try to extract the epoch from the header or a specific column
    nskiprows,epoch = extract_epoch_from_header(file)
   
    if epoch is None:
        raise ValueError(f"Could not find epoch for file: {file}")
   
    # Load the actual data (skipping header if necessary)
    df = pd.read_csv(file, skiprows=nskiprows,sep="\\s+")  # Adjust skiprows based on where the data starts
   
    # Convert 'time' column to a datetime, assuming it's in days
    # Adjust time column to start from the epoch time found in the file
    df['starttime_dt'] = pd.to_timedelta(df['starttime'], unit='D') + pd.Timestamp(epoch)
   
    # Return the adjusted DataFrame
    return df

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedeed)
rh_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240920_1134\\*.nas")
print(rh_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in rh_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
rh_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(rh_df)

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedded)
temperature_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240920_1140\\*.nas")
print(temperature_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in temperature_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
temperature_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(temperature_df)

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedded)
temperature_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240923_1438\\*.nas")
print(temperature_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in temperature_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
p_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(p_df)

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedeed)
wind_s_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240920_1137\\*.nas")
print(wind_s_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in wind_s_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
wind_s_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(wind_s_df)

In [ ]:
df=temperature_df
df ["RH"] = rh_df.rH
df ["flag_RH"] = rh_df.flag_rH
df ["flag_P"] = p_df.flag_P
df ["P"] = p_df.P
df ["flag_wind_s"] = wind_s_df.flag_wind_s
df ["wind_s"] = wind_s_df.wind_s
df

In [ ]:
#remove lines depending on flags
df=temperature_df
df ["RH"] = rh_df.rH
df ["flag_RH"] = rh_df.flag_rH
df ["flag_P"] = p_df.flag_P
df ["P"] = p_df.P
df ["flag_wind_s"] = wind_s_df.flag_wind_s
df ["wind_s"] = wind_s_df.wind_s
df = df[df.flag_T != 0.999]
df = df[df.flag_RH != 0.999]
df = df[df.flag_P != 0.999]
df = df[df.flag_wind_s != 0.999]
df = df[df.flag_T != 0.899]
df = df[df.flag_RH != 0.899]
df = df[df.flag_P != 0.899]
df = df[df.flag_wind_s != 0.899]
df = df[df.flag_wind_s != 0.456]
df = df[df.flag_T != 0.260]
df = df[df.flag_RH != 0.260]
df = df[df.flag_P != 0.260]
df = df[df.flag_wind_s != 0.260]
df = df[df.flag_T != 0.259]
df = df[df.flag_RH != 0.259]
df = df[df.flag_P != 0.259]
df = df[df.flag_wind_s != 0.259]
df = df[df.flag_T != 0.256]
df = df[df.flag_RH != 0.256]
df = df[df.flag_P != 0.256]
df = df[df.flag_wind_s != 0.256]
df.index = df["starttime_dt"]
#remove useless columns
df = df.drop(columns=["flag_P"])
df = df.drop(columns=["flag_RH"])
df = df.drop(columns=["flag_T"])
df = df.drop(columns=["flag_wind_s"])
df = df.drop(columns=["starttime"])
df = df.drop(columns=["endtime"])
df

In [ ]:
#loop for rounding to the hours
df["starttime_dt"] = [i.round("H") for i in df["starttime_dt"]]
df

In [37]:
# helper columns with  hour, month, year
df['hour'] = df['starttime_dt'].dt.hour
df['month'] = df['starttime_dt'].dt.month
df['year'] = df['starttime_dt'].dt.year
df['date'] = df['starttime_dt'].dt.date

In [ ]:
#dewpoint from T and RH
dewpoint=dewpoint_from_relative_humidity(df["T"].values*units.degC, df["RH"].values*units.percent)
dewpoint

In [ ]:
#Specific humidity from temperature and dewpoint
SH=specific_humidity_from_dewpoint(df["P"].values*units.hPa, dewpoint).to('g/kg')
df ["SH"] = SH
df

In [40]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Function to determine season
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [41]:
#season
seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
month_to_season = dict(zip(range(1,13), seasons))
month_to_season 
S=df.starttime_dt.dt.month.map(month_to_season) 
df['season']=S


In [ ]:

#Extract year and season
df['year'] = df['date'].dt.year
df['season'] = df['date'].apply(get_season)

# Group by year and season and calculate mean value (or sum, etc.)
grouped = df.groupby(['season', 'year'])['SH'].mean().unstack()

# Reorder the seasons
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
grouped = grouped.reindex(season_order)

# Plot for different years
grouped.plot(kind='line', marker='o', figsize=(10, 6))

# Customize plot
plt.title('Value by Season for Different Years')
plt.ylabel('SH [g/kg]')
plt.xlabel('Season')
plt.xticks(range(len(season_order)), season_order)
plt.legend(title='Year')
plt.grid(True)

# Show plot
plt.show()

In [43]:
#Filter the data to include only rows where the second column's values fall between the 25th and 75th percentiles.
#Plot the boxplot for the filtered data.

#import pandas as pd
#import matplotlib.pyplot as plt

# Example DataFrame
#data = {
 #   'column1': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],  # Column to filter (for percentiles)
  #  'column2': [5, 15, 25, 35, 45, 55, 65, 75, 85, 95]     # Column to plot
#}


# Step 1: Calculate the 25th and 75th percentiles of 'column1'
q25_ws = df['wind_s'].quantile(0.25)
q75_ws = df['wind_s'].quantile(0.75)
q25_SH = df['SH'].quantile(0.25)
q75_SH = df['SH'].quantile(0.75)

# Step 2: PBL: Filter the Data to include only rows where WS values are <= 25th and SH >= 75th percentiles
PBL_df = df[(df['wind_s'] <= q25_ws) & (df['SH'] >= q75_SH)]
# Step 2: FT (free troposphere): Filter the Data to include only rows where WS values are >= q75_ws and SH<= q25_SH percentiles
FT_df = df[(df['wind_s'] >= q75_ws) & (df['SH'] <= q25_SH)]

#descriptive statistics of PBL and FT regime occurrance in 24h and month
#PBL_df=(df.groupby(['month']).agg({'SH': ['mean','min', 'max', 'std', 'count'] }))

# Step 4: Create a boxplot for 'column2' using the filtered DataFrame
#plt.boxplot(PBL_df['column2'])
#plt.title('Boxplot of Column2 for Values in PBL')
#plt.ylabel('Values in Column2')
#plt.show()

In [ ]:
PBL_df.groupby(['month']).agg({'SH': ['mean','min', 'max', 'std', 'count'] })

In [ ]:
FT_df.groupby(['month']).agg({'SH': ['mean','min', 'max', 'std', 'count'] })

In [46]:
# Step 1: Calculate the 25th and 75th percentiles of 'column1'
q25_ws = df['wind_s'].quantile(0.25)
q75_ws = df['wind_s'].quantile(0.75)
q25_SH = df['SH'].quantile(0.25)
q75_SH = df['SH'].quantile(0.75)

In [ ]:
wind_s=df['wind_s']
SH=df['SH']
def _conditions3(wind_s , SH):
    rank = (     
          "PBL" if (wind_s <= q25_ws) & (SH >= q75_SH)
    else  "FT"  if (wind_s >= q75_ws) & (SH <= q25_SH)
    else  "Und"  
    )
    
    return rank
# Vectorize the function
func = np.vectorize(_conditions3)
# Create a new column based on the function
df["Atmos"] = func(wind_s, SH)
df

In [ ]:
#occurrence in each hour
# Step 2: Group by 'Month' and 'Flag' and count the occurrences
flag_counts = df.groupby(['hour', 'Atmos']).size().reset_index(name='Count')

# Step 3: Plot the scatter plot using seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(data=flag_counts, x='hour', y='Count', hue='Atmos', s=100)
plt.show()

In [ ]:
# Step 2: Group by 'hour' and Flag in 'Atmos' and count the occurrences
flag_counts = df.groupby(['hour', 'Atmos']).size().reset_index(name='Count')

# Step 3: Calculate the total number of items in the DataFrame (total flags)
total_count = len(df)

# Step 4: Calculate the percentage for each combination of Month and Flag in 'Atmos'
flag_counts['Percentage'] = (flag_counts['Count'] / total_count) * 100

# Display the result
print(flag_counts[['hour', 'Atmos', 'Count', 'Percentage']])

# Display the result

plt.figure(figsize=(8, 6))
sns.scatterplot(data=flag_counts, x='hour', y=flag_counts['Percentage'], hue='Atmos', s=100)
plt.show()

In [ ]:
#“FT”, PBL-representative data in 24h
# Step 2: Group by 'hour' and Flag in 'Atmos' and count the occurrences
flag_counts = df.groupby(['hour', 'Atmos']).size().reset_index(name='Count')

# Step 3: Calculate total counts per each flag across all hours
total_flag_counts = df.groupby('Atmos').size().reset_index(name='Total Count')

# Step 4: Merge the flag counts with the total counts
flag_counts = pd.merge(flag_counts, total_flag_counts, on='Atmos')

# Step 5: Calculate the percentage for each combination of Month and Flag based on the total count per flag
flag_counts['Percentage'] = (flag_counts['Count'] / flag_counts['Total Count']) * 100

# Display the result
print(flag_counts[['hour', 'Atmos', 'Count', 'Total Count', 'Percentage']])

plt.figure(figsize=(8, 6))
sns.scatterplot(data=flag_counts, x='hour', y='Percentage', hue='Atmos', s=100)
plt.show()

In [ ]:
#clacluate percentage of PBL occurence between 10 and 15 out of the PBL occurance h24
filtered_df = flag_counts[(flag_counts['Atmos'] == 'PBL') & (flag_counts['hour'].isin([10, 11, 12, 13, 14, 15]))]

# Step 7: Sum the percentages
total_percentage_PBL = filtered_df['Percentage'].sum()

# Display the result
print(total_percentage_PBL)


In [ ]:
#clacluate percentage of FT occurence between 20 and 6 out of the PBL occurance h24
filtered_df = flag_counts[(flag_counts['Atmos'] == 'FT') & (flag_counts['hour'].isin([20, 21, 22, 23, 0, 1,  2, 3, 4, 5, 6]))]

# Step 7: Sum the percentages
total_percentage_FT = filtered_df['Percentage'].sum()

# Display the result
print(total_percentage_FT)


In [ ]:
#occurrence in each month 
# Step 2: Group by 'Month' and 'Flag' and count the occurrences
flag_counts_m = df.groupby(['month', 'Atmos']).size().reset_index(name='Count')

# Step 3: Plot the scatter plot using seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(data=flag_counts_m, x='month', y='Count', hue='Atmos', s=100)
plt.show()

In [ ]:
##how flags are distributed across months in relation to the entire dataset
# Step 2: Group by 'Month' and 'Flag in 'Atmos'' and count the occurrences
flag_counts_m = df.groupby(['month', 'Atmos']).size().reset_index(name='Count')

# Step 3: Calculate the total number of items in the DataFrame (total flags)
total_count = len(df)

# Step 4: Calculate the percentage for each combination of Month and Flag in 'Atmos'
flag_counts_m['Percentage'] = (flag_counts_m['Count'] / total_count) * 100

# Display the result
print(flag_counts_m[['month', 'Atmos', 'Count', 'Percentage']])

# Display the result

plt.figure(figsize=(8, 6))
sns.scatterplot(data=flag_counts_m, x='month', y=flag_counts_m['Percentage'], hue='Atmos', s=100)
plt.show()